In [113]:
import random, csv
import numpy as np
import cPickle
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import networkx as nx
from collections import defaultdict
from sklearn.cluster import KMeans

### Reading test and train data, and node info

In [2]:
#nltk.download('punkt') # for tokenization
#nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()
stemmer_new = nltk.stem.SnowballStemmer("english")
with open("testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]

In [191]:
with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]

In [4]:
with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

IDs = [element[0] for element in node_info]

In [192]:
featfile = open("embeddings_new_1000.emd", 'r')
featline = featfile.readlines()
print featline[1]

9407087 -0.411383 -0.020357 -0.400334 -0.339862 -0.135672 -0.253496 0.186777 -0.343075 0.006982 -0.133885 -0.226440 0.221926 -0.095856 0.026859 -0.541065 -0.032277 0.158725 -0.163885 -0.013826 0.255544 0.155507 -0.306157 0.159946 0.193400 -0.024856 0.263089 0.044932 -0.202525 -0.395658 0.246506 0.039258 0.118137 -0.087199 -0.158658 0.039355 0.102043 0.151044 -0.160539 -0.147107 -0.027377 0.116614 -0.097601 0.091300 -0.048201 0.139412 -0.223881 -0.001322 -0.385248 0.019185 0.042991 -0.054089 -0.078147 -0.397054 -0.093546 -0.327359 0.443369 0.268929 -0.053761 -0.198621 -0.074796 0.303797 -0.048937 0.078628 -0.357489 0.221393 -0.000290 -0.204928 -0.210304 -0.105123 0.081045 -0.004886 -0.089612 0.194011 -0.103109 -0.069539 -0.005624 -0.016682 0.108760 -0.089497 0.109126 -0.620871 0.299417 -0.029055 0.018739 -0.064403 0.246944 0.135511 -0.104967 -0.086793 -0.091969 0.025420 0.352654 0.065260 -0.134090 0.025244 0.412901 -0.147076 0.105749 0.247415 0.004214 0.017703 -0.205978 -0.215702 -0.122

In [193]:
node_embeds = []
node_nums = []
for line in featline[1:]:
    line = line.strip().split(" ")
    node_embeds.append(np.array(line[1:], dtype=float))
    node_nums.append(line[0])
nodes_vec = np.array(node_embeds)  
print np.shape(nodes_vec)

(27770L, 128L)


In [206]:
k = 20
kmeans = KMeans(n_clusters=k, n_jobs=-1).fit(nodes_vec)

In [207]:
clusters_pred = kmeans.predict(nodes_vec).tolist()

In [208]:
cluster_node_pred = dict(zip(node_nums, clusters_pred))

In [218]:
file=open("features_train_unscaled.csv",'r')
lines=file.readlines()
feat = []
self_cit = []
counter = 0
for line in training_set:
    #line=line.strip().split(",")  
    #feat.append(line)
    source = line[0]
    target = line[1]
    
    feat.append([line[2], int(cluster_node_pred[source] == cluster_node_pred[target])])
    counter += 1
    if counter % 100000 == True:
        print counter, "training examples processsed"

1 training examples processsed
100001 training examples processsed
200001 training examples processsed
300001 training examples processsed
400001 training examples processsed
500001 training examples processsed
600001 training examples processsed


In [219]:
feat[0]

['1', 0]

### Creating features for the entire data to store in csv file
##### since this is very slow, I am writing only 50000 examples for now

In [211]:
## delete - just trying out this feature before writing into file
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
X = np.array(feat)[:, 3:]
#y = train_features[:,2]
y = np.array(feat)[:,2]
y = np.array(y, dtype=int)
print X.shape, y.shape

(615512L, 1L) (615512L,)


In [212]:
clf = RandomForestClassifier()#, max_features=2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)
print X_train.shape, X_val.shape, y_train.shape, y_val.shape
clf.fit(X_train, y_train)


(412393L, 1L) (203119L, 1L) (412393L,) (203119L,)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [213]:
y_val_pred = clf.predict(X_val)
f1score = f1_score(y_val, y_val_pred)
acc = accuracy_score(y_val, y_val_pred)
print 'F1-score: ', f1score
print 'Accuracy: ', acc

F1-score:  0.714486049098
Accuracy:  0.738384887677


In [220]:
# For running kmeans with different values of k and computing F1 score
for k in [20]:
    kmeans = KMeans(n_clusters=k, n_jobs=-1).fit(nodes_vec)
    clusters_pred = kmeans.predict(nodes_vec).tolist()
    cluster_node_pred = dict(zip(node_nums, clusters_pred))
    counter = 0
    for line in training_set:
        source = line[0]
        target = line[1]
    
        feat.append([line[2], int(cluster_node_pred[source] == cluster_node_pred[target])])
        counter += 1
        #if counter % 100000 == True:
            #print counter, "training examples processsed"

    X = np.array(feat)[:, 1:]
    y = np.array(feat)[:,0]
    y = np.array(y, dtype=int)

    clf = RandomForestClassifier()#, max_features=2)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)
    #print X_train.shape, X_val.shape, y_train.shape, y_val.shape
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict(X_val)
    f1score = f1_score(y_val, y_val_pred)
    acc = accuracy_score(y_val, y_val_pred)
    print 'k: ', k, ', F1-score: ', f1score, ', Accuracy: ', acc

k:  20 , F1-score:  0.713800205419 , Accuracy:  0.737976260222
